In [ ]:
pip install fxcmpy==1.2.9

In [ ]:
pip install python-socketio==4.4

In [ ]:
pip install python-engineio==3.9

In [ ]:
pip install websocket-client==1.1.0

In [1]:
import fxcmpy
import socketio
import datetime as dt
import math

In [2]:
TOKEN = '4ffd2f8fb79858912386a852640c9ba7ce68cbdd'

In [3]:
con = fxcmpy.fxcmpy(access_token=TOKEN, log_level='error', log_file=None)

In [ ]:
#configuration
globalSpreadBuy = 3
globalSpreadSell = 14
globalStop = 2

#global data
globalPositionOpen = 0
globalPositionTrailing = 0
globalPositionStop = 0
globalPositionType = ""
globalGross = 0
globalPrice = 0
globalMaxGross = 0

In [ ]:
def priceConvert(price):
    price = format(price, '.5f')
    if len(str(price)) < 6 :
        price =  int(str(price) + "0")
        
    return int(str(str(price).replace(".", ""))[:6])

In [ ]:
def priceConvertDot(price):
    return str(price)[:1] + "." + str(price)[1:]

In [ ]:
def on_tick(data, df):
    #print("price changed")
    #print("Ask: {}".format(data['Rates'][0]))
    activePosition = getActivePosition(data['Rates'][0])
    print(activePosition)
    #if activePosition != null
     #   print(activePosition)

In [ ]:
def getDifference(openPrice, actualPrice, isBuy):
    difference = 0
    openPrice = int(str(str(openPrice).replace(".", ""))[:6])
    actualPrice = int(str(str(actualPrice).replace(".", ""))[:6])
    
    if len(str(actualPrice)) < 6 :
        actualPrice =  int(str(actualPrice) + "0")
    elif len(str(openPrice)) < 6 :
        openPrice =  int(str(openPrice) + "0")    
    
    if isBuy: 
        difference = actualPrice - openPrice
    else:
        difference = openPrice - actualPrice
    
    return difference

In [ ]:
def closeAllPositions():
    con.close_all()
    global globalMaxGross
    global globalSpreadBuy
    global globalSpreadSell
    global globalPositionType
    global globalGross
    global globalPrice
    global globalPositionOpen
    global globalPositionStop
    global globalPositionTrailing
    globalMaxGross = 0
    globalPositionType = ""
    globalGross = 0
    globalPrice = 0
    globalPositionOpen = 0
    globalPositionStop = 0
    globalPositionTrailing = 0

In [ ]:
def getActivePosition(price):
    global globalMaxGross
    global globalSpreadBuy
    global globalSpreadSell
    global globalPositionType
    global globalGross
    global globalPrice
    global globalPositionOpen
    global globalPositionStop
    global globalPositionTrailing
    
    globalPrice =  priceConvert(price)
    
    trades = con.get_open_trade_ids()
    
    if not trades:
        return "Price % s" % (priceConvertDot(globalPrice))
    else:    
        pos = con.get_open_position(trades[0])  
        globalGross = pos.get_grossPL()
        globalPositionOpen = priceConvert(pos.get_open())        
                
        #difference = getDifference(pos.get_open(), price, pos.get_isBuy())    
        
        if pos.get_isBuy():
            globalPositionType = "Buy"
            globalPositionTrailing = globalPositionOpen + globalSpreadBuy
            
            if globalPrice > globalPositionTrailing:
                globalPositionTrailing = globalPrice
                if (globalPositionTrailing - globalStop) > globalPositionStop:
                    globalPositionStop = globalPositionTrailing - globalStop
            
            if globalPositionStop > 0:
                if globalPrice < globalPositionStop:
                    print("Closed at: " + str(globalGross))
                    closeAllPositions()
        else:
            globalPositionType = "Sell"
            globalPositionTrailing = globalPositionOpen - globalSpreadSell
            
            if globalPrice < globalPositionTrailing:
                globalPositionTrailing = globalPrice
                
                if (globalPositionTrailing + globalStop) < globalPositionStop or globalPositionStop == 0: 
                    globalPositionStop = globalPositionTrailing + globalStop
        
            if globalPositionStop > 0:
                if globalPrice > globalPositionStop:
                    print("Closed at: " + str(globalGross))
                    closeAllPositions()
                
        return "% s | Gross % s | Stop % s | Trailing % s | Actual % s | Open % s" % ( globalPositionType, globalGross, priceConvertDot(globalPositionStop), priceConvertDot(globalPositionTrailing), priceConvertDot(globalPrice), priceConvertDot(globalPositionOpen))

In [ ]:
con.subscribe_market_data('EUR/USD',(on_tick,))

In [ ]:
con.unsubscribe_market_data('EUR/USD')

In [ ]:
con.get_accounts().T

In [ ]:
con.get_last_price('EUR/USD')

In [ ]:
sel = ['tradeId', 'amountK', 'currency', 'grossPL', 'isBuy']

In [ ]:
con.get_open_positions()[sel]

In [ ]:
con.get_open_positions_summary().T

In [ ]:
con.get_closed_positions_summary().T

In [ ]:
order = con.create_market_buy_order('EUR/USD',10)

In [ ]:
order = con.create_market_sell_order('EUR/USD',10)

In [ ]:
opciones = ['grossPL']

In [ ]:
pos = con.get_open_position(140265045)

In [ ]:
#print(pos.get_grossPL())
print(pos)

In [ ]:
pos.close()

In [ ]:
con.close_all()
globalMaxGross = 0
globalPositionType = ""
globalGross = 0
globalPrice = 0
globalPositionOpen = 0
globalPositionStop = 0
globalPositionTrailing = 0

In [ ]:
con.get_closed_positions().T

In [ ]:
class ActivePosition:
  def __init__(self, typePosition, gross, openPrice, actualPrice, maxGross, difference):
    self.typePosition = typePosition
    self.gross = gross
    self.openPrice = openPrice
    self.actualPrice = actualPrice
    self.maxGross = maxGross
    self.difference = difference
  def __str__(self):
    return "% s | Gross % s | Open % s | Actual % s | accumulated % s | difference % s" % (self.typePosition, self.gross, self.openPrice, format(self.actualPrice, '.5f'), self.maxGross, self.difference)    

In [ ]:
globalMaxGross = 0
globalPositionType = ""
globalGross = 0
globalPrice = 0
globalPositionOpen = 0
globalPositionStop = 0
globalPositionTrailing = 0